### Mit Interaktionsvariablen arbeiten (Kapitel 3.5.2)
#### 1) Daten laden und Datumsindex einstellen

In [3]:
import pandas as pd
from math import ceil

pd.set_option('display.max_columns', 20)

data_url=r'https://github.com/tplusone/hanser_ml_zeitreihen/blob/master/Daten/trip_weather_hourly.csv?raw=true'
df = pd.read_csv(data_url)
df['start_date'] = pd.to_datetime(df['start_date'])
df = df.set_index('start_date')
df.head()

number_trips     duration  max_temperature_f  \
start_date                                                          
2013-08-29 09:00:00            12  1022.500000               74.0   
2013-08-29 10:00:00            14  1677.857143               74.0   
2013-08-29 11:00:00            42  2203.809524               74.0   
2013-08-29 12:00:00           120   934.833333               74.0   
2013-08-29 13:00:00            87  3934.103448               74.0   

                     mean_temperature_f  min_temperature_f  max_dew_point_f  \
start_date                                                                    
2013-08-29 09:00:00                68.0               61.0             61.0   
2013-08-29 10:00:00                68.0               61.0             61.0   
2013-08-29 11:00:00                68.0               61.0             61.0   
2013-08-29 12:00:00                68.0               61.0             61.0   
2013-08-29 13:00:00                68.0               61.0             61.0   

                     mean_dew_point_f  min_dew_point_f  max_humidity  \
start_date                                                             
2013-08-29 09:00:00              58.0             56.0          93.0   
2013-08-29 10:00:00              58.0             56.0          93.0   
2013-08-29 11:00:00              58.0             56.0          93.0   
2013-08-29 12:00:00              58.0             56.0          93.0   
2013-08-29 13:00:00              58.0             56.0          93.0   

                     mean_humidity  ...  mean_visibility_miles  \
start_date                          ...                          
2013-08-29 09:00:00           75.0  ...                   10.0   
2013-08-29 10:00:00           75.0  ...                   10.0   
2013-08-29 11:00:00           75.0  ...                   10.0   
2013-08-29 12:00:00           75.0  ...                   10.0   
2013-08-29 13:00:00           75.0  ...                   10.0   

                     min_visibility_miles  max_wind_Speed_mph  \
start_date                                                      
2013-08-29 09:00:00                  10.0                23.0   
2013-08-29 10:00:00                  10.0                23.0   
2013-08-29 11:00:00                  10.0                23.0   
2013-08-29 12:00:00                  10.0                23.0   
2013-08-29 13:00:00                  10.0                23.0   

                     mean_wind_speed_mph  max_gust_speed_mph  \
start_date                                                     
2013-08-29 09:00:00                 11.0                28.0   
2013-08-29 10:00:00                 11.0                28.0   
2013-08-29 11:00:00                 11.0                28.0   
2013-08-29 12:00:00                 11.0                28.0   
2013-08-29 13:00:00                 11.0                28.0   

                     precipitation_inches  cloud_cover  events  \
start_date                                                       
2013-08-29 09:00:00                     0          4.0     NaN   
2013-08-29 10:00:00                     0          4.0     NaN   
2013-08-29 11:00:00                     0          4.0     NaN   
2013-08-29 12:00:00                     0          4.0     NaN   
2013-08-29 13:00:00                     0          4.0     NaN   

                     wind_dir_degrees  zip_code  
start_date                                       
2013-08-29 09:00:00             286.0     94107  
2013-08-29 10:00:00             286.0     94107  
2013-08-29 11:00:00             286.0     94107  
2013-08-29 12:00:00             286.0     94107  
2013-08-29 13:00:00             286.0     94107  

[5 rows x 25 columns]

#### 2) Features aus Datum extrahieren

In [4]:
df['weekday'] = df.index.weekday
df['month'] = df.index.month
df['hour'] = df.index.hour
df[['weekday', 'month', 'hour']].head()

weekday  month  hour
start_date                               
2013-08-29 09:00:00        3      8     9
2013-08-29 10:00:00        3      8    10
2013-08-29 11:00:00        3      8    11
2013-08-29 12:00:00        3      8    12
2013-08-29 13:00:00        3      8    13

#### 2) Train-Test-Split durchführen

In [5]:
from sklearn.model_selection import train_test_split

X = df[['month', 'weekday', 'hour']]
y = df['number_trips']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, 
                                            random_state=11)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14066, 3), (3517, 3), (14066,), (3517,))

#### 3) Interaktionsvariable aus Weekday und hour bilden 
- Zunächst hour und weekday in One-Hot bzw. Dummy-Format bringen

In [12]:
from sklearn.preprocessing import OneHotEncoder

# Daten extrahieren
X_train_hour = X_train[['hour']]
X_test_hour = X_test[['hour']]

# One-Hot-Encoding
ohe = OneHotEncoder(sparse=False)
ohe.fit(X_train_hour)
X_train_hour = ohe.transform(X_train_hour)
X_test_hour = ohe.transform(X_test_hour)

# Dummy-Variable aus weekday erzeugen (Wochentage=0, Wochenende=1)
X_train_wd = X_train['weekday'].map(lambda x:
                                1 if x >= 5 else 0)
X_test_wd = X_test['weekday'].map(lambda x:
                                1 if x >= 5 else 0)
# Array in 2D-Format bringen [[1,3,6,4]]

X_train_wd = X_train_wd.values.reshape(-1, 1)
X_test_wd = X_test_wd.values.reshape(-1, 1)
X_train_wd


array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int64)

- Interaktionsvariable erzeugen und mit einfachen Variablen zusammenführen

In [14]:
import numpy as np

# Interaktionsvariablen erzeugen
y = c1*x1+c2*x2+c3*x3+c4*x3*x2.....
X_train_int = X_train_wd * X_train_hour
X_test_int = X_test_wd * X_test_hour

# Variablen zusammenfügen
X_train_compl = np.concatenate(
    [X_train_wd, X_train_hour, X_train_int], 
    axis=1)
X_test_compl = np.concatenate(
    [X_test_wd, X_test_hour, X_test_int], 
    axis=1)
X_train_compl.shape, X_test_compl.shape

((14066, 49), (3517, 49))

#### 4) Regression anlernen

In [15]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.fit(X_train_compl, y_train)
r2 = linear.score(X_test_compl, y_test)
print('r2 {:.3f}'.format(r2))

r2 0.829
